In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.ml.feature import VectorAssembler,StringIndexer
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator

In [2]:
spark

In [3]:
sc

<SparkContext master=spark://devenv:7077 appName=PySparkShell>

In [6]:
data = spark.read.csv("hdfs://devenv/user/ml_datasets/movies/ratings.csv",
                     inferSchema=True,
                     header=True)

In [8]:
# Split data into train and test sets
train_data, test_data = data.randomSplit([0.8,0.2])

In [9]:
# Model training
als = ALS(maxIter=5,userCol="userId",itemCol="movieId",ratingCol="rating" , coldStartStrategy="drop")
model = als.fit(train_data)

In [10]:
model

ALSModel: uid=ALS_b54ef83428c6, rank=10

In [11]:
# 對所有使用者推薦產品
# Specify the number of movies you would like to recommand for each user
user_movies = model.recommendForAllUsers(3)
user_movies.show(100, truncate=False)
# # The users who are most likely to like a particular movie 對商品推薦給使用者
# movie_uers = model.recommendForAllItems(3)
# movie_uers.show(100, truncate=False)

/Users/henglingyeh/spark-3.2.1-bin-hadoop2.7/python/pyspark/sql/context.py:125: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


+------+------------------------------------------------------------+
|userId|recommendations                                             |
+------+------------------------------------------------------------+
|1     |[{3153, 4.417581}, {106438, 4.0929856}, {850, 3.9530582}]   |
|2     |[{501, 4.6661973}, {1162, 4.5458536}, {3417, 4.453329}]     |
|3     |[{501, 4.641822}, {2267, 4.483765}, {97957, 4.462822}]      |
|4     |[{83318, 6.16349}, {83411, 6.16349}, {83359, 6.16349}]      |
|5     |[{2563, 5.215541}, {5114, 5.164263}, {59684, 5.0076714}]    |
|6     |[{83318, 4.9932995}, {83411, 4.9932995}, {83359, 4.9932995}]|
|7     |[{83318, 5.169519}, {83411, 5.169519}, {83359, 5.169519}]   |
|8     |[{83318, 4.9807477}, {83411, 4.9807477}, {83359, 4.9807477}]|
|9     |[{83318, 4.9756203}, {83411, 4.9756203}, {83359, 4.9756203}]|
|10    |[{25852, 5.0704403}, {8121, 5.0704403}, {4612, 5.0704403}]  |
|11    |[{119145, 5.632885}, {106438, 5.565439}, {3153, 5.5317583}] |
|12    |[{98491, 4.8

In [12]:
# Transform the test data using the model to get predictions
predicted_test_data = model.transform(test_data)
# Evalute model performance with test set
evaluator = RegressionEvaluator(predictionCol="prediction", labelCol="rating", metricName="rmse")
print("rmse: {}".format(evaluator.evaluate(predicted_test_data)))

rmse: 0.9141769616692828


最後將推薦資料存到DB，當使用者登入時就可以使用，每天更新模型